In [1]:
import pandas as pd

#google cloud big query libaries
from google.cloud import bigquery
from google.oauth2 import service_account

import os

In [2]:
dirShiny = "E:/GitHub/Resources/R-Shiny/HTS-Trip-Lengths-2012/input"

group_by_columns = ['groupRegionName','groupWorkers','groupVehicles','groupTripPurpose','groupMainMode','binSize']
display(group_by_columns)

index_columns = group_by_columns + ['binStart']
display(index_columns)

['groupRegionName',
 'groupWorkers',
 'groupVehicles',
 'groupTripPurpose',
 'groupMainMode',
 'binSize']

['groupRegionName',
 'groupWorkers',
 'groupVehicles',
 'groupTripPurpose',
 'groupMainMode',
 'binSize',
 'binStart']

# Setup Data

In [3]:
# Link to BigQuery Client through API

key_path = r"C:\Users\bhereth\confidential-2023-utah-hts-db5335615978.json"
#key_path = r"C:\Users\bhereth\tdm-scenarios-a85044dbbfd3.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
print ('Successfully Linked to BigQuery Client!')

Successfully Linked to BigQuery Client!


In [4]:
str_sql = """
SELECT
  bins.binSize,
  bins.binStart,
  gSs.groupRegionName,
  gNw.groupWorkers,
  gNv.groupVehicles,
  gMtb.groupMainMode,
  gTt.groupTripPurpose,
  COUNT(*) as numTripRecords,
  SUM(trip.weight) AS sumTripWeight
FROM
  `confidential-2023-utah-hts.bins.binsDist` as bins,
  `confidential-2023-utah-hts.previous_hts_2012.hh` as hh,
  `confidential-2023-utah-hts.previous_hts_2012.trip` as trip,
  `confidential-2023-utah-hts.previous_hts_2012_groupings.groupRegionName` as gSs,
  `confidential-2023-utah-hts.previous_hts_2012_groupings.groupWorkers` as gNw,
  `confidential-2023-utah-hts.previous_hts_2012_groupings.groupVehicles` as gNv,
  `confidential-2023-utah-hts.previous_hts_2012_groupings.groupMainMode` as gMtb,
  `confidential-2023-utah-hts.previous_hts_2012_groupings.groupTripPurpose` as gTt
WHERE
  trip.trip_distance >= bins.binStart AND
  trip.trip_distance < bins.binStart + bins.binSize AND
  hh.regionname = gSs.regionname AND
  hh.password = trip.password AND
  hh.workers = gNw.workers AND
  hh.num_vehicles = gNv.num_vehicles AND
  trip.trip_purpose = gTt.trip_purpose AND
  trip.main_mode = gMtb.main_mode
GROUP BY
  bins.binSize,
  bins.binStart,
  gSs.groupRegioNname,
  gNw.groupWorkers,
  gNv.groupVehicles,
  gTt.groupTripPurpose,
  gMtb.groupMainMode
"""

In [5]:
# test connection
df_trips_by_distance_bins = client.query(str_sql).to_dataframe()

display(df_trips_by_distance_bins)

,binSize,binStart,groupRegionName,groupWorkers,groupVehicles,groupMainMode,groupTripPurpose,numTripRecords,sumTripWeight
0,0.5,10.5,WFRC-MAG,2,-1,-1,All,214,23720.105692
1,0.5,10.5,WFRC-MAG,2,-1,-1,HBW,81,9310.725310
2,0.5,10.5,WFRC-MAG,2,-1,1,All,201,21865.341736
3,0.5,10.5,WFRC-MAG,2,-1,1,HBW,72,8076.682396
4,0.5,10.5,WFRC-MAG,2,2,-1,All,116,11495.167885
...,...,...,...,...,...,...,...,...,...
236910,1.0,15.0,WFRC-MAG,-1,1,1,HBSch,2,466.210662
236911,1.0,15.0,All,1,1,-1,HBSch,2,466.210662
236912,1.0,15.0,All,1,1,1,HBSch,2,466.210662
236913,1.0,15.0,All,-1,1,-1,HBSch,2,466.210662


# Calculate Distributions

In [6]:
# Group by 'trip_type' and 'distance_miles_2mibin' and sum 'trip_weight_sum'
df_grouped = df_trips_by_distance_bins.groupby(index_columns).agg({'numTripRecords': 'sum', 'sumTripWeight': 'sum'}).reset_index()

# Calculate the total weight for each trip type
total_weights = df_grouped.groupby(group_by_columns)['sumTripWeight'].transform('sum')

# Calculate percentage distribution
df_grouped['pctTripWeight'] = (df_grouped['sumTripWeight'] / total_weights) * 100

# Add cumulative distribution
df_grouped['cumPctTripWeight'] = df_grouped.groupby(group_by_columns)['pctTripWeight'].cumsum()

df_grouped = df_grouped.fillna(0)

display(df_grouped)

,groupRegionName,groupWorkers,groupVehicles,groupTripPurpose,groupMainMode,binSize,binStart,numTripRecords,sumTripWeight,pctTripWeight,cumPctTripWeight
0,All,-1,-1,All,-1,0.5,0.0,9959,9.699015e+05,10.399190,10.399190
1,All,-1,-1,All,-1,0.5,0.5,11173,1.111671e+06,11.919230,22.318420
2,All,-1,-1,All,-1,0.5,1.0,9257,8.675964e+05,9.302285,31.620705
3,All,-1,-1,All,-1,0.5,1.5,8034,7.705527e+05,8.261792,39.882496
4,All,-1,-1,All,-1,0.5,2.0,6922,6.662265e+05,7.143217,47.025713
...,...,...,...,...,...,...,...,...,...,...,...
236910,WFRC-MAG,6,5,NHBNW,2,0.5,12.5,1,2.457866e+02,50.000000,50.000000
236911,WFRC-MAG,6,5,NHBNW,2,0.5,13.0,1,2.457866e+02,50.000000,100.000000
236912,WFRC-MAG,6,5,NHBNW,2,1.0,12.0,1,2.457866e+02,50.000000,50.000000
236913,WFRC-MAG,6,5,NHBNW,2,1.0,13.0,1,2.457866e+02,50.000000,100.000000


In [7]:
df_grouped.to_csv('../input/hts-trip-lengths.csv', index=False)